In [1]:
import SimPEG as simpeg
import simpegMT as simpegmt
import numpy as np
import matplotlib.pyplot as plt

In [2]:
## Setup the modelling
# Setting up 1D mesh and conductivity models to forward model data.

# Frequency
nFreq = 31
freqs = np.logspace(3,-3,nFreq)
# Set mesh parameters
ct = 20
air = simpeg.Utils.meshTensor([(ct,16,1.4)])
core = np.concatenate( (  np.kron(simpeg.Utils.meshTensor([(ct,10,-1.3)]),np.ones((5,))) , simpeg.Utils.meshTensor([(ct,5)]) ) )
bot = simpeg.Utils.meshTensor([(core[0],10,-1.4)])
x0 = -np.array([np.sum(np.concatenate((core,bot)))])
# Make the model
m1d = simpeg.Mesh.TensorMesh([np.concatenate((bot,core,air))], x0=x0)

# Setup model varibles
active = m1d.vectorCCx<0.
layer1 = (m1d.vectorCCx<-500.) & (m1d.vectorCCx>=-800.)
layer2 = (m1d.vectorCCx<-3500.) & (m1d.vectorCCx>=-5000.)
# Set the conductivity values
sig_half = 2e-3
sig_air = 1e-8
sig_layer1 = .2
sig_layer2 = .2
# Make the true model
sigma_true = np.ones(m1d.nCx)*sig_air
sigma_true[active] = sig_half
sigma_true[layer1] = sig_layer1
sigma_true[layer2] = sig_layer2
# Extract the model 
m_true = np.log(sigma_true[active])
# Make the background model
sigma_0 = np.ones(m1d.nCx)*sig_air
sigma_0[active] = sig_half
m_0 = np.log(sigma_0[active])

# Set the mapping
actMap = simpeg.Maps.ActiveCells(m1d, active, np.log(1e-8), nC=m1d.nCx)
mappingExpAct = simpeg.Maps.ExpMap(m1d) * actMap

In [3]:
## Setup the layout of the survey, set the sources and the connected receivers

# Receivers 
rxList = []
for rxType in ['z1dr','z1di']:
    rxList.append(simpegmt.SurveyMT.RxMT(simpeg.mkvc(np.array([0.0]),2).T,rxType))
# Source list
srcList =[]
for freq in freqs:
        srcList.append(simpegmt.SurveyMT.srcMT_polxy_1Dprimary(rxList,freq))
# Make the survey
survey = simpegmt.SurveyMT.SurveyMT(srcList)
survey.mtrue = m_true
# Set the problem
problem = simpegmt.ProblemMT1D.eForm_psField(m1d,sigmaPrimary=sigma_0,mapping=mappingExpAct)
from pymatsolver import MumpsSolver
problem.solver = MumpsSolver
problem.pair(survey)

In [4]:
## Read the data
std = 0.05 # 5% std
# Load the files if they exist
if os.path.isfile('MT1D_dtrue.npy') and os.path.isfile('MT1D_dobs.npy'):
    d_true = np.load('MT1D_dtrue.npy')
    d_obs = np.load('MT1D_dobs.npy')
else:
    # Forward model
    d_true = survey.dpred(m_true)
    np.save('MT1D_dtrue.npy',d_true)
    d_obs = d_true + (std*abs(d_true)*np.random.randn(*d_true.shape))
    np.save('MT1D_dobs.npy',d_obs)
# Assign the datas to the survey object
survey.dtrue = d_true
survey.dobs = d_obs
survey.std = np.abs(survey.dobs*std) + 0.01*np.linalg.norm(survey.dobs) #survey.dobs*0 + std
# Assign the data weight
survey.Wd = 1/survey.std #(abs(survey.dobs)*survey.std)

In [5]:
## Setup the inversion proceedure

# Define a counter
C =  simpeg.Utils.Counter()
# Set the optimization
opt = simpeg.Optimization.InexactGaussNewton(maxIter = 30)
opt.counter = C
opt.LSshorten = 0.5
opt.remember('xc')
# Data misfit
dmis = simpeg.DataMisfit.l2_DataMisfit(survey)
# Regularization
# Note: We want you use a mesh the corresponds to the domain we want to solve, the active cells.
if True:
    regMesh = simpeg.Mesh.TensorMesh([m1d.hx[problem.mapping.sigmaMap.maps[-1].indActive]],m1d.x0)
    reg = simpeg.Regularization.Tikhonov(regMesh)
else:
    reg = simpeg.Regularization.Tikhonov(m1d,mapping=mappingExpAct)
reg.smoothModel = True
reg.alpha_s = 1e-9
reg.alpha_x = 1.
# reg.alpha_xx = 0.001
# Inversion problem
invProb = simpeg.InvProblem.BaseInvProblem(dmis, reg, opt)
invProb.counter = C
# Beta cooling
beta = simpeg.Directives.BetaSchedule()
betaest = simpeg.Directives.BetaEstimate_ByEig(beta0_ratio=0.75)
saveModel = simpeg.Directives.SaveModelEveryIteration()
saveModel.fileName = 'Inversion_NoStoppingregMesh_smoothTrue'
# Create an inversion object
inv = simpeg.Inversion.BaseInversion(invProb, directiveList=[beta,betaest,saveModel]) 


In [6]:
# Run the inversion, given the background model as a start.
mopt = inv.run(m_0)

SimPEG.InvProblem will set Regularization.mref to m0.
SimPEG.InvProblem is setting bfgsH0 to the inverse of the eval2Deriv.
                    ***Done using same solver as the problem***
SimPEG.l2_DataMisfit is creating default weightings for Wd.
SimPEG.SaveModelEveryIteration will save your models as: '###-Inversion_NoStoppingregMesh_smoothTrue.npy'
============================ Inexact Gauss Newton ============================
  #     beta     phi_d     phi_m       f      |proj(x-g)-x|  LS    Comment   
-----------------------------------------------------------------------------
   0  2.08e+05  2.18e+05  0.00e+00  2.18e+05    4.01e+04      0              
   1  2.08e+05  2.52e+04  2.37e-04  2.52e+04    6.20e+03      0              
   2  2.08e+05  3.46e+03  3.27e-04  3.53e+03    1.03e+03      0   Skip BFGS  
   3  2.60e+04  1.72e+03  2.64e-04  1.73e+03    2.61e+02      0   Skip BFGS  
   4  2.60e+04  1.03e+03  1.19e-02  1.34e+03    2.44e+02      0   Skip BFGS  
   5  2.60e+04  6.85e

In [7]:
modList = []
modFiles = glob('*Inversion_NoStoppingregMesh_smoothTrue.npy')
modFiles.sort()
for f in modFiles:
    modList.append(np.load(f))

In [8]:
# simpegmt.Utils.dataUtils.plotMT1DModelData(problem,modList)
# plt.show()


In [9]:
# %matplotlib qt
# fig= simpegmt.Utils.dataUtils.plotMT1DModelData(problem,modList[-3:-1])
# fig.suptitle('No stopping-useMref ')
# plt.show()

In [10]:
%matplotlib qt
fig = simpegmt.Utils.dataUtils.plotMT1DModelData(problem,[m_0,mopt])
fig.suptitle('No stopping-useMref')
plt.show()

/home/gudni/anaconda/lib/python2.7/site-packages/numpy/ma/core.py:2834: FutureWarning: Numpy has detected that you (may be) writing to an array returned
by numpy.diagonal or by selecting multiple fields in a record
array. This code will likely break in a future numpy release --
see numpy.diagonal or arrays.indexing reference docs for details.
The quick fix is to make an explicit copy (e.g., do
arr.diagonal().copy() or arr[['f0','f1']].copy()).
  if (obj.__array_interface__["data"][0]
/home/gudni/anaconda/lib/python2.7/site-packages/numpy/ma/core.py:2835: FutureWarning: Numpy has detected that you (may be) writing to an array returned
by numpy.diagonal or by selecting multiple fields in a record
array. This code will likely break in a future numpy release --
see numpy.diagonal or arrays.indexing reference docs for details.
The quick fix is to make an explicit copy (e.g., do
arr.diagonal().copy() or arr[['f0','f1']].copy()).
  != self.__array_interface__["data"][0]):


In [11]:
plt.show()

In [12]:
1/np.exp(mopt)

array([  4.26403069e+01,   3.00486580e+02,   1.66374083e+02,
         7.56749193e+00,   1.95232762e+00,   1.28591697e+01,
         9.25428204e+01,   1.58941963e+02,   1.23328303e+02,
         7.51647545e+01,   4.59620120e+01,   2.84873346e+01,
         1.67854087e+01,   9.65288171e+00,   5.69417961e+00,
         4.11415753e+00,   4.04091270e+00,   5.89702878e+00,
         1.03685013e+01,   1.49615226e+01,   1.78450685e+01,
         2.21298842e+01,   3.14894964e+01,   5.03867813e+01,
         8.57783978e+01,   1.39225191e+02,   2.10683526e+02,
         3.13430053e+02,   4.54348237e+02,   6.35619644e+02,
         8.20077602e+02,   9.86597201e+02,   1.13311026e+03,
         1.22681391e+03,   1.23177188e+03,   1.13462480e+03,
         9.82900015e+02,   7.79370193e+02,   5.53800277e+02,
         3.42690620e+02,   1.91257904e+02,   1.00287868e+02,
         4.40936077e+01,   1.57081219e+01,   4.84819824e+00,
         2.37649055e+00,   2.81483584e+00,   7.91239926e+00,
         3.26193232e+01,